In [1]:
!pip install yfinance 
!pip install bs4
!pip install nbformat
!pip install html5lib



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import sys
print(sys.executable)

/opt/anaconda3/envs/myenv/bin/python


In [ ]:
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install yfinance
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install pdfplumber

In [ ]:
import yfinance as yf
import pandas as pd
import requests
import time
import re
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import defaultdict


In [ ]:
def new_graph(stock_data, revenue_data, stock):
    fig = new_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [ ]:
tesla=yf.Ticker('TSLA')

In [ ]:
tesla_data = tesla.history(period='max')


In [ ]:
tesla_data.reset_index(inplace=True)
tesla_data

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re


CIK = '0001318605'
submissions_url = f'https://data.sec.gov/submissions/CIK{CIK}.json'
headers = {'User-Agent': 'Mozilla/5.0 (compatible; your-email@example.com)'}  # Set real email

# Get recent filings (2017-now)
resp = requests.get(submissions_url, headers=headers)
data = resp.json()
filings = data['filings']['recent']

entries = []
for form, acc_no, file_name, date_filed in zip(
    filings['form'], filings['accessionNumber'], filings['primaryDocument'], filings['filingDate']
):
    if form in ('10-Q', '10-K'):
        acc_no_clean = acc_no.replace('-', '')
        url = f'https://www.sec.gov/Archives/edgar/data/{CIK.strip("0")}/{acc_no_clean}/{file_name}'
        entries.append((form, date_filed, url))

# Add old entries (2010–2016) manually
old_entries = [
    # 2010
    ('10-Q', '2010-08-13', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312510188792/d10q.htm'),
    ('10-Q', '2010-11-12', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312510259068/d10q.htm'),
    ('10-K', '2011-03-03', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312511054847/d10k.htm'),

    # 2011
    ('10-Q', '2011-05-13', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312511139677/d10q.htm'),
    ('10-Q', '2011-08-12', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312511221497/d10q.htm'),
    ('10-Q', '2011-11-14', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312511308489/d226201d10q.htm'),
    ('10-K', '2012-02-27', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312512081990/d279413d10k.htm'),

    # 2012
    ('10-Q', '2012-05-10', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312512225825/d325967d10q.htm'),
    ('10-Q', '2012-08-02', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312512332138/d364775d10q.htm'),
    ('10-Q', '2012-11-07', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312512457610/d410318d10q.htm'),
    ('10-K', '2013-03-07', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312513096241/d452995d10k.htm'),

    # 2013
    ('10-Q', '2013-05-10', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312513212354/d511008d10q.htm'),
    ('10-Q', '2013-08-09', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312513327916/d549636d10q.htm'),
    ('10-Q', '2013-11-08', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312513435480/d588506d10q.htm'),
    ('10-K', '2014-02-26', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312514069681/d668062d10k.htm'),

    # 2014
    ('10-Q', '2014-05-09', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312514192606/d715897d10q.htm'),
    ('10-Q', '2014-08-08', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312514303175/d766922d10q.htm'),
    ('10-Q', '2014-11-07', 'https://www.sec.gov/Archives/edgar/data//1318605/000119312514403635/d812482d10q.htm'),
    ('10-K', '2015-02-26', 'https://www.sec.gov/Archives/edgar/data//1318605/000156459015001031/tsla-10k_20141231.htm'),

    # 2015
    ('10-Q', '2015-05-08', 'https://www.sec.gov/Archives/edgar/data//1318605/000156459015003789/tsla-10q_20150331.htm'),
    ('10-Q', '2015-08-07', 'https://www.sec.gov/Archives/edgar/data//1318605/000156459015006666/tsla-10q_20150630.htm'),
    ('10-Q', '2015-11-05', 'https://www.sec.gov/Archives/edgar/data//1318605/000156459015009741/tsla-10q_20150930.htm'),
    ('10-K', '2016-02-24', 'https://www.sec.gov/Archives/edgar/data//1318605/000156459016013195/tsla-10k_20151231.htm'),

    # 2016
    ('10-Q', '2016-05-10', 'https://www.sec.gov/Archives/edgar/data//1318605/000156459016018886/tsla-10q_20160331.htm'),
    ('10-Q', '2016-08-05', 'https://www.sec.gov/Archives/edgar/data//1318605/000156459016023024/tsla-10q_20160630.htm'),
    ('10-Q', '2016-11-01', 'https://www.sec.gov/Archives/edgar/data//1318605/000156459016026820/tsla-10q_20160930.htm'),
    ('10-K', '2017-03-01', 'https://www.sec.gov/Archives/edgar/data//1318605/000156459017003118/tsla-10k_20161231.htm')
]

entries += old_entries
entries.sort(key=lambda x: x[1])  # sort by date_filed

# Scrape revenue from filings
revenue_by_year = defaultdict(dict)  # {'2012': {'Q1': ..., 'Q2': ..., 'Q3': ..., 'Q4': ...}}


# These are taken directly from Tesla's SEC filings
known_values = {
    # Format: (year, period): value in millions
    ('2019', 'FY'): 24578,  
    ('2020', 'FY'): 31536,  
    ('2021', 'FY'): 53823,  
    ('2022', 'FY'): 81462,  
    ('2023', 'FY'): 96773,  
    ('2024', 'FY'): 97690,
    
}


def parse_revenue_from_url(url, year, period):
    # First check if we have known value for this year/period
    if (year, period) in known_values:
        #print(f"Using known value for {year} {period}: {known_values[(year, period)]} million")
        return known_values[(year, period)]
    
    resp = requests.get(url, headers=headers)
    if resp.content.strip().startswith(b'<?xml'):
        soup = BeautifulSoup(resp.content, 'xml')
    else:
        soup = BeautifulSoup(resp.content, 'html.parser')
    
    # Try to find the unit (in thousands or millions)
    unit_multiplier = 1  # Default multiplier (no change)
    
    # Look for statements indicating the unit in the document
    unit_text = soup.find(string=re.compile(r'(in thousands|in millions)', re.IGNORECASE))
    if unit_text:
        if re.search(r'in thousands', unit_text, re.IGNORECASE):
            unit_multiplier = 0.001  # Convert thousands to millions
    
    # If we couldn't find it in general text, look in table descriptions
    if unit_multiplier == 1:
        for table in soup.find_all('table'):
            table_text = table.get_text()
            if re.search(r'in thousands', table_text, re.IGNORECASE):
                unit_multiplier = 0.001
                break
    
    # search for the revenue
    for table in soup.find_all('table'):
        if 'Total revenues' in table.get_text():
            for row in table.find_all('tr'):
                cells = row.find_all(['td', 'th'])
                if len(cells) >= 2:
                    label = cells[0].get_text(strip=True).lower()
                    if 'total revenues' in label:
                        for cell in cells[1:]:
                            text_value = cell.get_text(strip=True)
                            clean_value = text_value.replace('$', '')
                            
                            if re.search(r'\d', clean_value):
                                try:
                                    # Handle commas as thousand separators
                                    clean_value = clean_value.replace(',', '')
                                    # Convert to float only if it looks valid
                                    if re.match(r'^\d+(\.\d+)?$', clean_value):
                                        numeric_value = float(clean_value)
                                    
                                    # Check for plausibility because a recent year values could be large
                                    filing_year = int(year)
                                    
                                    # Apply the unit multiplier
                                    final_value = numeric_value * unit_multiplier
                                    
                                    # Additional validation based on year
                                    if filing_year >= 2019 and period == 'FY' and final_value < 1000:
                                        # This is likely a case of misinterpreted commas
                                        #print(f"Warning: {year} {period} value ({final_value}) seems too small.")
                                        #print(f"Original value: {clean_value}, numeric: {numeric_value}")
                                        
                                        # If the raw value has a decimal point after the first few digits
                                        # it might be a comma-as-decimal issue
                                        if '.' in clean_value and len(clean_value.split('.')[0]) <= 3:
                                            corrected = final_value * 1000
                                            #print(f"  Adjusting to: {corrected} million")
                                            final_value = corrected
                                    
                                    
                                    return final_value
                                    
                                except ValueError:
                                    contnue
        


for form, date_filed, url in entries:
    if form == '10-K':
        year = str(int(date_filed[:4]) - 1)  # Adjust for FY
        period = 'FY'
    elif form == '10-Q':
        year = date_filed[:4]
        month = int(date_filed[5:7])
        if month <= 5:
            period = 'Q1'
        elif month <= 8:
            period = 'Q2'
        else:
            period = 'Q3'
    else:
        continue

    #print(f"Processing {form} for {year} {period} at {url}")
    revenue = parse_revenue_from_url(url, year, period)
    if revenue:
        #print(f"  Found revenue: {revenue} million")
        revenue_by_year[year][period] = revenue
    else:
        revenue_by_year[year][period] = None
    time.sleep(1)  # Be nice to the SEC server

# Print detailed information about the data
#print("\n=== Revenue Data by Year ===")
for year in sorted(revenue_by_year.keys()):
    quarters = revenue_by_year[year]
    #print(f"\nYear {year}:")
    for period in ['Q1', 'Q2', 'Q3', 'Q4', 'FY']:
        if period in quarters:
            continue
            #print(f"  {period}: {quarters[period]} million")

# Calculate Q4 values where missing
#print("\n=== Computing Q4 values where missing ===")
for year, revs in revenue_by_year.items():
    if 'FY' in revs and all(q in revs for q in ('Q1', 'Q2', 'Q3')) and 'Q4' not in revs:
        try:
            q1 = revs['Q1']
            q2 = revs['Q2']
            q3 = revs['Q3']
            fy = revs['FY']
            
            # Ensure all are valid numbers
            if all(isinstance(val, (int, float)) for val in [q1, q2, q3, fy]) and all(val is not None for val in [q1, q2, q3, fy]):
                q4 = fy - (q1 + q2 + q3)
                # Only set if Q4 is not negative or suspicious
                if q4 >= 0:
                    revenue_by_year[year]['Q4'] = round(q4, 2)
                    #print(f"  Calculated Q4 for {year}: {q4:.2f} million")
                else:
                    #print(f"  Invalid Q4 calculation for {year}: {q4:.2f} million (negative)")
                    revenue_by_year[year]['Q4'] = None
            else:
                #print(f"  Cannot calculate Q4 for {year}: missing valid quarterly data")
                revenue_by_year[year]['Q4'] = None
        except Exception as e:
            #print(f"  Error calculating Q4 for {year}: {e}")
            revenue_by_year[year]['Q4'] = None

# Sanity checks
#print("\n=== Sanity Checks ===")
for year in sorted(revenue_by_year.keys()):
    quarters = revenue_by_year[year]
    if 'FY' in quarters and all(q in quarters for q in ['Q1', 'Q2', 'Q3']):
        # Fix: Filter out None values before summing
        q_values = [quarters.get(q, 0) for q in ['Q1', 'Q2', 'Q3']]
        q_values = [v for v in q_values if v is not None]  # Filter out None values
        
        if len(q_values) == 3:  # Only if we have all 3 quarters with valid values
            q_sum = sum(q_values)
            q4 = quarters.get('Q4', 0)
            if q4 is None:
                q4 = 0
                
            fy = quarters['FY']
            if fy is not None:
                total_sum = q_sum + q4
                ratio = fy / total_sum if total_sum > 0 else 0
                
                #print(f"Year {year}: FY({fy}) / Sum({total_sum}) = {ratio:.2f}")
                
                if abs(ratio - 1.0) > 0.1:  # More than 10% difference
                    print(f"  WARNING: Possible discrepancy in {year} data")
                    #print(f"    Q1: {quarters.get('Q1', 'N/A')}")
                    #print(f"    Q2: {quarters.get('Q2', 'N/A')}")
                    #print(f"    Q3: {quarters.get('Q3', 'N/A')}")
                    #print(f"    Q4: {quarters.get('Q4', 'N/A')}")
                    #print(f"    FY: {quarters.get('FY', 'N/A')}")
            else:
                continue
                #print(f"Year {year}: FY value is None, skipping sanity check")
        else:
            continue
            #print(f"Year {year}: Missing valid quarterly data, skipping sanity check")

# Convert to DataFrame
all_results = []
for year, quarters in revenue_by_year.items():
    for q in ['Q1', 'Q2', 'Q3', 'Q4', 'FY']:
        if q in quarters:
            all_results.append({'year': year, 'quarter': q, 'revenue_in_millions': quarters[q]})

df = pd.DataFrame(all_results).sort_values(by=['year', 'quarter'])
df['revenue_in_millions'] = df['revenue_in_millions'].apply(lambda x: round(x, 2) if pd.notnull(x) else None)

# Export to CSV
df.to_csv('tesla_quarterly_revenue_millions.csv', index=False)

print("\n=== Final Data ===")
print(df)

## Annual Revenue of TESLA from 2010 to 2024


In [ ]:
fullYear_df = df[df['quarter']== 'FY'].copy()

fullYear_df=fullYear_df.drop(columns=['quarter'])
fullYear_df.reset_index(drop=True)

In [ ]:
!git add .
!git commit -m "first commit from Jupyter Notebook"
!git push -u origin main